In [ ]:
import numpy as np 
import pandas as pd
import random
import seaborn as sns
from matplotlib import pyplot as plt
import tensorflow as tf


np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# Load the dataset
data1 = pd.read_csv('V1_23_Feb_180m_anomalous_data(gaussian).csv')
dataframe = data1

dataframe['datetimestamp'] = pd.to_datetime(dataframe['datetimestamp'])

# Take only 'datetimestamp', 'Hz_mod_anomaly', and 'mod_BIN' columns
df = dataframe[['datetimestamp', 'Hz_mod_anomaly', 'mod_BIN']]
df.set_index('datetimestamp', inplace=True)

plt.figure(figsize=(10, 6)) 
plt.plot(df.index, df['Hz_mod_anomaly'], label='Hz_mod_anomaly', color='blue')

plt.xticks(rotation=45)
plt.gcf().set_facecolor('white')
plt.xlabel('Date Timestamp')
plt.ylabel('Hz Mod Anomaly')
plt.title('Anomalous Hz Over Time')
plt.grid(True)
plt.legend()
plt.show()

print(df.head())

In [ ]:
# print start and end date
print("start date is:", df.index.min())
print("end date is:", df.index.max())
# Now we have to choose train and test sets 
# We will train LSTM autoencoders using only normal dataset which is labeled as 0 in the dataframe we have to separate normal from anomalous data.

# Separate normal data (label 0) from anomalous data
normal_data = df[df['mod_BIN'] == 0]
anomalous_data = df[df['mod_BIN'] != 0]

# Print start and end dates of normal data
print("Start date of normal data is:", normal_data.index.min())
print("End date of normal data is:", normal_data.index.max())

# Print start and end dates of anomalous data
print("Start date of anomalous data is:", anomalous_data.index.min())
print("End date of anomalous data is:", anomalous_data.index.max())


In [ ]:
# Now we make train and test dataset
# in this case 80 % of normal data would be used for training and rest 20% of normal data + all anomalous data would be used in test 

start_anomalous_date = anomalous_data.index.min()

# Split normal data into train before anomaly and train after anomaly
train_before_anomaly = normal_data[normal_data.index < start_anomalous_date]
train_after_anomaly = normal_data[normal_data.index >= start_anomalous_date]

# Calculate the number of rows for 80% of normal data
train_normal_size = int(0.8 * len(normal_data))

# Select the first 80% of normal data for training
train = normal_data.iloc[:train_normal_size]

# Select the remaining 20% of normal data for testing
test_normal = normal_data.iloc[train_normal_size:]

# Concatenate test normal data with anomalous data
test = pd.concat([test_normal, anomalous_data])

print("Shape of train:", train.shape)
print("Shape of test:", test.shape)

sns.set_style("whitegrid")
sns.set_context("notebook")

plt.figure(figsize=(10, 6))  # Adjust the figure size if needed
sns.lineplot(x='datetimestamp', y='Hz_mod_anomaly', data=train_before_anomaly, label='Train Data', color='blue')
sns.lineplot(x='datetimestamp', y='Hz_mod_anomaly', data=train_after_anomaly, color='blue')
sns.lineplot(x='datetimestamp', y='Hz_mod_anomaly', data=test_normal, color='red')
sns.lineplot(x='datetimestamp', y='Hz_mod_anomaly', data=anomalous_data, label='Test Data', color='red')
plt.xlabel('Date')
plt.ylabel('Hz_mod_anomaly')
plt.title('Train vs Test Data')
plt.legend()
plt.gca().set_facecolor('white')
plt.gcf().set_facecolor('white')

plt.show()


In [ ]:
# # Since LSTM use sigmoid and tanh so we need out values to be normalized , we will use Standard Scaler for that
# 
# scaler = StandardScaler()
# scaler = scaler.fit(train[['Hz_mod_anomaly']])
# 
# train.loc[:, 'Hz_mod_anomaly'] = scaler.transform(train[['Hz_mod_anomaly']])
# test.loc[:, 'Hz_mod_anomaly'] = scaler.transform(test[['Hz_mod_anomaly']])

# Drop the 'mod_BIN' column from train and test DataFrames because it was just labels to split between train and test 
train = train.drop(columns=['mod_BIN'])
test = test.drop(columns=['mod_BIN'])

# Display the first few rows of train and test sets
print(train.head())
print(test.head())

In [ ]:
seq_size = 20 # Number of time steps to look back 
# larger sequence size (look further back) may improve forecasting 

def to_sequence(x, y, seq_size=1):
    x_values = []
    y_values = []
    
    for i in range(len(x)-seq_size):
        #print(i)
        x_values.append(x.iloc[i:(i+seq_size)].values)
        y_values.append(y.iloc[i+seq_size])
        
    return np.array(x_values), np.array(y_values)

trainX, trainY = to_sequence(train[['Hz_mod_anomaly']], train['Hz_mod_anomaly'], seq_size)
testX, testY = to_sequence(test[['Hz_mod_anomaly']], test['Hz_mod_anomaly'], seq_size)

print("train X shape", trainX.shape)
print("train Y shape", trainY.shape)
print("test X shape", testX.shape)
print("test Y shape", testY.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, RepeatVector, TimeDistributed, Dense
model = Sequential()
model.add(LSTM(128, activation='tanh', recurrent_activation='sigmoid', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(64, activation='tanh', recurrent_activation='sigmoid', return_sequences=False))
model.add(RepeatVector(trainX.shape[1]))
model.add(LSTM(64, activation='tanh', recurrent_activation='sigmoid', return_sequences=True))
model.add(LSTM(128, activation='tanh', recurrent_activation='sigmoid', return_sequences=True))
model.add(TimeDistributed(Dense(trainX.shape[2])))
model.compile(optimizer='adam', loss='mae', metrics=["mape"])

# Compile model
model.compile(optimizer='adam', loss='mae', metrics=["mape"])
model.summary()

In [ ]:
import time

# Measure the time
start_training_time = time.time()
# Fit the model
history = model.fit(trainX, trainY, epochs=35, batch_size=100, validation_split=0.2, verbose=1)
end_training_time = time.time()
training_time = end_training_time - start_training_time
print(f"Total training time: {training_time:.2f} seconds")

In [ ]:
# training history
training_loss = history.history['loss']
training_mape = history.history['mape']
val_loss = history.history['val_loss']
val_mape = history.history['val_mape']
epochs = range(1, len(training_loss) + 1)

plt.figure(figsize=(10, 5))

# Plot training and validation loss 
plt.plot(epochs, training_loss, color='blue', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')

# Plot training and validation MAPE 
plt.plot(epochs, training_mape, color='orange', linestyle='--', label='Training MAPE')
plt.plot(epochs, val_mape, color='red', linestyle='--', label='Validation MAPE')

plt.title('Training and Validation Loss / MAPE')
plt.xlabel('Epochs')
plt.ylabel('Loss / MAPE')
plt.grid(True)
plt.legend()
plt.show()

# Extract the final loss and MAPE values
final_training_loss = training_loss[-1]
final_val_loss = val_loss[-1]
final_training_mape = training_mape[-1]
final_val_mape = val_mape[-1]

# Print the final loss and MAPE values
print("Final Training Loss:", final_training_loss)
print("Final Validation Loss:", final_val_loss)
print("Final Training MAPE:", final_training_mape)
print("Final Validation MAPE:", final_val_mape)


In [ ]:
# When reconstruction error (MAPE) is larger than the threshold which we set then there is anomaly

# Calculate MAE for training prediction
trainPredict = model.predict(trainX)
trainMAE = np.mean(np.abs(trainPredict - trainX), axis=1)
# Print the mean of test MAE
print("Mean of Train MAE:", np.mean(trainMAE))

# Plot histogram
plt.figure(figsize=(8, 6))
plt.hist(trainMAE, bins=30)
plt.xlabel('Mean Absolute Error (MAE)')
plt.ylabel('Frequency')
plt.title('Histogram of Mean Absolute Error (MAE) in Training Prediction')
plt.show()

# Calculate MAPE for each sample
trainActual = trainX  # Assuming trainX contains the actual values
trainMAPE = np.mean(np.abs(trainPredict - trainActual) / trainActual, axis=1) * 100

# Print the mean of MAPE
print("Mean of Train MAPE:", np.mean(trainMAPE))

# Plot histogram of MAPE
plt.figure(figsize=(8, 6))
plt.hist(trainMAPE, bins=30)
plt.xlabel('Mean Absolute Percentage Error (MAPE)')
plt.ylabel('Frequency')
plt.title('Histogram of Mean Absolute Percentage Error (MAPE) in Training Prediction')
plt.show()

In [ ]:
# Calculate reconstruction loss (MAE) for testing dataset
testPredict = model.predict(testX)
testMAE = np.mean(np.abs(testPredict - testX), axis=1)

# Print the mean of test MAE
print("Mean of Test MAE:", np.mean(testMAE))

# Plot histogram
plt.hist(testMAE, bins=30)
plt.xlabel('Test MAE')
plt.ylabel('Frequency')
plt.title('Histogram of Test MAE')
plt.show()

# Calculate MAPE for each sample
testActual = testX  # Assuming trainX contains the actual values
testMAPE = np.mean(np.abs(testPredict - testActual) / testActual, axis=1) * 100

# Print the mean of MAPE
print("Mean of Test MAPE:", np.mean(testMAPE))

# Plot histogram of MAPE
plt.figure(figsize=(8, 6))
plt.hist(testMAPE, bins=30)
plt.xlabel('Mean Absolute Percentage Error (MAPE)')
plt.ylabel('Frequency')
plt.title('Histogram of Mean Absolute Percentage Error (MAPE) in test Prediction')
plt.show()

In [ ]:
max_trainMAE = 0.76
max_trainMAPE = 0.1
# thresholding using MAPE 
anomaly_df = pd.DataFrame(test[seq_size:])
anomaly_df['testMAPE'] = testMAPE
anomaly_df['max_trainMAPE'] = max_trainMAPE
anomaly_df['anomaly'] = anomaly_df['testMAPE'] > anomaly_df['max_trainMAPE']
anomaly_df['Hz_mod_anomaly'] = test[seq_size:]['Hz_mod_anomaly']

# Plot test MAE vs max_trainMAE
sns.lineplot(x=anomaly_df.index, y=anomaly_df['testMAPE'])
sns.lineplot(x=anomaly_df.index, y=anomaly_df['max_trainMAPE'])
plt.xticks(rotation=45)

In [ ]:
# Extract anomalies DataFrame with 'anomaly' column equal to True
anomalies = anomaly_df.loc[anomaly_df['anomaly'] == True]
# 
# # Reshape the 'Hz_mod_anomaly' column to be a 2-dimensional array
# hz_mod_anomaly_reshaped = scaler.inverse_transform(anomaly_df['Hz_mod_anomaly'].values.reshape(-1, 1))
# 
# # Reshape the anomalies 'Hz_mod_anomaly' column to be a 2-dimensional array
# anomalies_reshaped = scaler.inverse_transform(anomalies['Hz_mod_anomaly'].values.reshape(-1, 1))
# 
# # Plot the data
# plt.figure(figsize=(10, 6))
# sns.lineplot(x=anomaly_df.index, y=hz_mod_anomaly_reshaped.flatten(), color='blue')  # Flatten to convert 2D array to 1D array
# sns.scatterplot(x=anomalies.index, y=anomalies_reshaped.flatten(), color='red')  # Flatten to convert 2D array to 1D array
# 
# # Show plot
# plt.show()

# Reshape the 'Hz_mod_anomaly' column to be a 2-dimensional array
hz_mod_anomaly_reshaped = anomaly_df['Hz_mod_anomaly'].values.reshape(-1, 1)

# Reshape the anomalies 'Hz_mod_anomaly' column to be a 2-dimensional array
anomalies_reshaped = anomalies['Hz_mod_anomaly'].values.reshape(-1, 1)

# Plot the data
plt.figure(figsize=(10, 6))
sns.lineplot(x=anomaly_df.index, y=hz_mod_anomaly_reshaped.flatten(), color='blue') 
sns.scatterplot(x=anomalies.index, y=anomalies_reshaped.flatten(), color='red')  

# Show plot
plt.show()


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.metrics import mean_absolute_percentage_error, confusion_matrix

# Load the dataset(s)
# Adjust the path to your dataset file
data1 = pd.read_csv('V3S3.csv')  # Replace 'V3S1.csv' with the correct file path if needed

# Combine the datasets (if you have more)
combined_data = pd.concat([data1], ignore_index=True)

# Preprocess the combined dataset
combined_data['datetimestamp'] = pd.to_datetime(combined_data['datetimestamp'])

# Plot the combined dataset
plt.figure(figsize=(10, 6))
sns.lineplot(x='datetimestamp', y='Hz_mod_anomaly', data=combined_data)
plt.xticks(rotation=45)
plt.gcf().set_facecolor('white') 
plt.xlabel('Date')
plt.ylabel('Frequency (Hz)')
plt.title('Testing Dataset Version 3 Signature 1')
plt.show()

# Convert the combined dataset into sequences
combined_X, combined_Y = to_sequence(combined_data[['Hz_mod_anomaly']], combined_data['Hz_mod_anomaly'], seq_size)

# Measure the time before starting the prediction
start_time = time.time()

# Use the trained model to predict the reconstruction errors (MAPE) on the combined dataset
combined_predict = model.predict(combined_X)

# Measure the time after prediction
end_time = time.time()

# Calculate total inference time
inference_time = end_time - start_time
print(f"Total inference time: {inference_time:.2f} seconds")

# Calculate combined MAPE
combined_mape = np.mean(np.abs(combined_predict - combined_X) / combined_X, axis=1) * 100

# Thresholding using MAPE
max_trainMAPE = 0.1

# Capture all details in a DataFrame for easy plotting
anomaly_df = pd.DataFrame(combined_data[seq_size:])
anomaly_df['combinedMAPE'] = combined_mape
anomaly_df['max_trainMAPE'] = max_trainMAPE
anomaly_df['anomaly'] = anomaly_df['combinedMAPE'] > max_trainMAPE
anomaly_df['Hz_mod_anomaly'] = combined_data[seq_size:]['Hz_mod_anomaly']

# Plot combined MAPE vs max_trainMAPE
plt.figure(figsize=(10, 6))
sns.lineplot(x=anomaly_df.index, y=anomaly_df['combinedMAPE'], label='MAPE')
sns.lineplot(x=anomaly_df.index, y=anomaly_df['max_trainMAPE'], label='Threshold', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Mean Absolute Percentage Error (MAPE)')
plt.title('Anomaly Detection on Testing Dataset (V3-S2)')
plt.legend()
plt.xticks(rotation=45)
plt.show()

# Identify anomalies based on the threshold
combined_anomalies = anomaly_df[anomaly_df['anomaly'] == True]

# Plot anomalies
plt.figure(figsize=(10, 6))
sns.lineplot(x=combined_data['datetimestamp'], y=combined_data['Hz_mod_anomaly'], color='blue', label='Normal Data')
sns.scatterplot(x=combined_anomalies['datetimestamp'], y=combined_anomalies['Hz_mod_anomaly'], color='red', label='Anomalies')
plt.xlabel('Date')
plt.ylabel('Frequency (Hz)')
plt.title('Anomaly Detection on Testing Dataset (V3-S2)')
plt.legend()
plt.xticks(rotation=45)
plt.show()

# Get the true labels
true_labels = (combined_data['mod_BIN'][seq_size:] != 0).astype(int)

# confusion matrix
conf_matrix = confusion_matrix(true_labels, combined_mape > max_trainMAPE)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Anomaly'], yticklabels=['Normal', 'Anomaly'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

TP = conf_matrix[1, 1]
FP = conf_matrix[0, 1]
TN = conf_matrix[0, 0]
FN = conf_matrix[1, 0]

# Print the counts
print("True Positives (Anomalies correctly predicted as anomalies):", TP)
print("False Positives (Normal data incorrectly predicted as anomalies):", FP)
print("True Negatives (Normal data correctly predicted as normal):", TN)
print("False Negatives (Anomalies incorrectly predicted as normal):", FN)

# Calculate Precision, Recall, and F1-score
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)


print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)
accuracy = (TP + TN) / (TP + FP + TN + FN)
print("Accuracy:", accuracy)
print(f"Total inference time: {inference_time:.2f} seconds")
